In [1]:
# process tagged data

import pandas as pd
from collections import Counter

df_niu_1 = pd.read_csv("Snow_Tagging/SnowTagging_Niu_ROUND4_1_tagged.csv")
df_cat_1 = pd.read_csv("Snow_Tagging/SnowTagging_Cat_ROUND4_1_tagged.csv")
df_kathy_1 = pd.read_csv("Snow_Tagging/SnowTagging_Kathy_ROUND4_1_tagged.csv")
df_niu = pd.read_csv("Snow_Tagging/SnowTagging_Niu_ROUND4_2_tagged.csv")
df_cat = pd.read_csv("Snow_Tagging/SnowTagging_Cat_ROUND4_2_tagged.csv")
df_kathy = pd.read_csv("Snow_Tagging/SnowTagging_Kathy_ROUND4_2_tagged.csv")

#merge 2 dfs
df_niu=pd.concat([df_niu_1,df_niu],axis=0)
df_cat=pd.concat([df_cat_1,df_cat],axis=0)
df_kathy=pd.concat([df_kathy_1,df_kathy],axis=0)

exclusion=[":mwOOXLC23BE", ":Dg1v4Yi9xN0", ":sxz-xvr-7cs", ":AOWs0WN6aNQ", ":ei4AoxZsA0M", ":IgGuIOGB2PI",
          ":qvSqdiW0Jl0", ":YwqcsYLCGyI", ":tPqwtz3gz_I", ":B4tfjOfa63Q"]
df_niu=df_niu.loc[~df_niu.videoId.isin(exclusion)].reset_index()
df_cat=df_cat.loc[~df_cat.videoId.isin(exclusion)].reset_index()
df_kathy=df_kathy.loc[~df_kathy.videoId.isin(exclusion)].reset_index()
assert(len(df_niu) == len(df_cat) == len(df_kathy))
df_cat

,index,embedUrl,videoId,title,isIncluded,diagnostic,mention,prognostic,motivational_action,motivational_rationale
0,0,https://www.youtube.com/embed/N2w4spsYm_Q,:N2w4spsYm_Q,Oppo Reno 2 Vs Oneplus 7 Pro Camera Comparison...,True,[none_apply],[embedded],[none_apply],[donation],[celebrity_effect]
1,1,https://www.youtube.com/embed/eh3bxXHqF2U,:eh3bxXHqF2U,Making a Particle Detector (Cloud Chamber),True,[none_apply],[hashtag],[creation],[none_apply],[none_apply]
2,2,https://www.youtube.com/embed/m7MOlhL6el4,:m7MOlhL6el4,#TeamTrees Custom Zombies LIVE! (ft. Zombie Ve...,True,[none_apply],[embedded],[gameplay],[donation],[tree_benefits;celebrity_effect;community_effect]
3,3,https://www.youtube.com/embed/xygBlTIv9U0,:xygBlTIv9U0,help us plant 20 MILLION trees! (#TeamTrees),True,[climate_change;deforestation;other_issues],[dedicated],[spread_the_word;environmental_discussion],[donation;environment_actions],[celebrity_effect;community_effect]
4,4,https://www.youtube.com/embed/exoFrIJwwV0,:exoFrIJwwV0,Updating Trees to be USEFUL in Minecraft! (#Te...,True,[none_apply],[embedded],[gameplay],[donation;support_channel],[celebrity_effect;community_effect]
...,...,...,...,...,...,...,...,...,...,...
95,55,https://www.youtube.com/embed/5sp87LcBZLo,:5sp87LcBZLo,How Much Air Can A Tree Hold? #TeamTrees,True,[climate_change],[dedicated],[spread_the_word;environmental_discussion],[donation;environment_actions],[tree_benefits;community_effect]
96,56,https://www.youtube.com/embed/iW4lHAYAO28,:iW4lHAYAO28,Every Tree Pokemon RANKED #TeamTrees,True,[none_apply],[embedded],[knowledge],[donation;more_trees],[celebrity_effect;community_effect]
97,57,https://www.youtube.com/embed/AIBeJGmGurs,:AIBeJGmGurs,"I Chopped 100,000 Trees in Forest Nothing #tea...",True,[none_apply],[embedded],[gameplay],[none_apply],[celebrity_effect]
98,58,https://www.youtube.com/embed/KTrliR_wTUU,:KTrliR_wTUU,Killing the ender dragon in hardcore with a sa...,True,[none_apply],[embedded],[donation;gameplay],[none_apply],[none_apply]


In [6]:
import nltk
from nltk.metrics import agreement,masi_distance,jaccard_distance,binary_distance
from nltk.metrics.agreement import AnnotationTask
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
import numpy as np

class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# cosine similarity is assuming full rank matrices, won't work for rank-deficient
# go through dimension reduction for sparse matrix
def cosine_distance(vec_a,vec_b):
#     vec_a, vec_b = list(vec_a), list(vec_b)
#     print(type(vec_a))
    distance = spatial.distance.cosine(vec_a,vec_b)
    return distance

# dropping columns with all 0s in the matrix
def dim_reduction(vector):
    vector = np.array(vector)
    idx_zero_cols = np.argwhere(np.all(vector[..., :] == 0, axis=0))
    res = np.delete(vector, idx_zero_cols, axis=1)
    return res

def toList(object):
    taglist = object[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return taglist

# format of task_data into calculate_score_cosine and calculate_score_jaccard is different
def calculate_score_cosine(question, task_data):
    distance_fn = binary_distance if question=="mention" else cosine_distance
    cosine_task = AnnotationTask(data=task_data,distance = distance_fn)
    print(f"Fleiss's Kappa using Cosine distance: {cosine_task.multi_kappa()}")
#     print(f"Krippendorff's Alpha using Cosine distance: {cosine_task.alpha()}")

def calculate_score_jaccard(question, task_data):
    distance_fn = binary_distance if question=="mention" else jaccard_distance
    jaccard_task = AnnotationTask(data=task_data,distance = distance_fn)
    print(f"Fleiss's Kappa using Jaccard distance: {jaccard_task.multi_kappa()}")
    print(f"Krippendorff's Alpha using Jaccard distance: {jaccard_task.alpha()}")

    
# for each row in df_cat:
# take other rows in dfniu and dfkathy, turn into dict for each row of each df
# combine them into a list -> go through mlb_fit
# loop through array and create annotation set of (coder, video, array_row) -> add to task_data 

def generate_task_data_cosine(question, coder_dict):
    mlb = MultiLabelBinarizer()
    if not coder_dict:
        return
    df_seed = list(coder_dict.values())[0]
    task_data = []
    for index, row in df_seed.iterrows():
        answer_list = []
        for coder, df_coder in coder_dict.items():
            answer_list.append(set(toList(df_coder.iloc[index][question])))
        arr = dim_reduction(mlb.fit_transform(answer_list))
        for pair in zip([list(r) for r in arr],coder_dict.keys()):
            answer, coder = pair
            annotation = coder, row["videoId"], tuple(answer)
            task_data.append(annotation)
    return task_data

def generate_task_data_jaccard(question, coder_dict):
    task_data = []
    for coder, df_coder in coder_dict.items():
        for index, row in df_coder.iterrows():
            annotation = toAnnotation(coder,row,question)
            task_data.append(annotation)
    return task_data

def toFrozenSet(object):
    taglist = object[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return frozenset(taglist)

def toAnnotation(coder, row, question):
    item = row["videoId"]
    tagging = toFrozenSet(row[question])
    return coder, item, tagging
        

In [7]:
CODER_DICT = {"cat":df_cat, "niu":df_niu, "kathy":df_kathy}
QUESTION = ["diagnostic",
             "mention",
             "prognostic",
             "motivational_action",
             "motivational_rationale"]

for question in QUESTION:
    print(color.BOLD + question.upper() + color.END)
    task_data_cosine = generate_task_data_cosine(question, CODER_DICT)
    task_data_jaccard = generate_task_data_jaccard(question, CODER_DICT)
    calculate_score_cosine(question, task_data_cosine)
    calculate_score_jaccard(question, task_data_jaccard)

DIAGNOSTIC
Fleiss's Kappa using Cosine distance: 0.6263940258623947
Fleiss's Kappa using Jaccard distance: 0.530719803624287
Krippendorff's Alpha using Jaccard distance: 0.5189773163216058
MENTION
Fleiss's Kappa using Cosine distance: 0.5484194681384845
Fleiss's Kappa using Jaccard distance: 0.5642437813956301
Krippendorff's Alpha using Jaccard distance: 0.5580555099359501
PROGNOSTIC
Fleiss's Kappa using Cosine distance: 0.6408164154633531
Fleiss's Kappa using Jaccard distance: 0.6987872569910295
Krippendorff's Alpha using Jaccard distance: 0.6897707862317691
MOTIVATIONAL_ACTION
Fleiss's Kappa using Cosine distance: 0.7149217669619486
Fleiss's Kappa using Jaccard distance: 0.6277675399941245
Krippendorff's Alpha using Jaccard distance: 0.5509782488064154
MOTIVATIONAL_RATIONALE
Fleiss's Kappa using Cosine distance: 0.7617002339826381
Fleiss's Kappa using Jaccard distance: 0.6961087789226704
Krippendorff's Alpha using Jaccard distance: 0.6221707622411734


In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics import agreement,masi_distance,jaccard_distance
from scipy import spatial
import numpy as np


mlb = MultiLabelBinarizer()
arr = mlb.fit_transform([{'none_apply'}, {'climate','forest'} , {'fr'}])
list(mlb.classes_)
names = ["a","b","c"]

a = frozenset([0, 0, 1, 0])
a
# np.array(a)


from nltk.metrics.agreement import AnnotationTask

# cosine similarity is assuming full rank matrices, won't work for rank-deficient
# go through dimension reduction for sparse matrix
def cosine_distance(vec_a,vec_b):
    vec_a, vec_b = list(vec_a), list(vec_b)
    distance = spatial.distance.cosine(vec_a,vec_b)
    return distance

task_data = [('coder1','video0',tuple([1,0,0])),
            ('coder2','video0',tuple([1,0,1])),
            ('coder3','video0',tuple([1,1,0])),
            ('coder1','video1',tuple([1])),
            ('coder2','video1',tuple([1])),
            ('coder3','video1',tuple([1]))]
cosine_task = AnnotationTask(data=task_data,distance = cosine_distance)
cosine_task.multi_kappa()

# a = [1,2,5]
# b = []
# b.append(i for i in a)
# b

0.7587141249717989